# Private LLMs on Your Local Machine
(adapting the manual from https://betterprogramming.pub/private-llms-on-local-and-in-the-cloud-with-langchain-gpt4all-and-cerebrium-6dade79f45f6)

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
# Define text colors
green = "\033[0;32m"
white = "\033[0;39m"
cyan  = "\033[36m"

In [ ]:
template = """
You are a friendly chatbot assistant that responds in a conversational
manner to users questions. Keep the answers short, unless specifically
asked by the user to elaborate on something.

Question: {question}

Answer:"""

In [4]:
prompt = PromptTemplate(template=template, input_variables=["question"])

## Local LLM *(e.g., GTP4all)*

In [5]:
llm_local = GPT4All(
    model='../Models/ggml-gpt4all-j-v1.3-groovy.bin',
    callbacks=[StreamingStdOutCallbackHandler()]
)

Found model file at  ./Models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './Models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [8]:
question_2 = input("Question? ")
llm_local(question_2)


The sentence is not clear. It's difficult to understand what the speaker means by saying that they live in Denmark or if it has any relation with their current situation. Can you please provide more context or clarify your question?

"\nThe sentence is not clear. It's difficult to understand what the speaker means by saying that they live in Denmark or if it has any relation with their current situation. Can you please provide more context or clarify your question?"

## Remote LLM via API call *(e.g., OpenAI )*

In [7]:
from langchain.chat_models import ChatOpenAI

# Accessing the OPENAI KEY
import environ
env = environ.Env()
environ.Env.read_env()
API_KEY = env('OPENAI_API_KEY')

# Simple LLM call Using LangChain
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=API_KEY)


/home/drphyl/.cache/pypoetry/virtualenvs/local_llms-CM_XZ3EH-py3.10/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_187161/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


In [29]:
query = "Which language is used to create chatgpt?"
response = llm(query)
print(f"{green}{query}")
print(f"{cyan}ChatGPT: {white}{response}")

Which language is used to create chatgpt?
ChatGPT: ChatGPT is primarily developed using the Python programming language. The underlying model architecture is based on OpenAI's GPT (Generative Pre-trained Transformer) model, which is implemented using Python and frameworks like TensorFlow or PyTorch.


## Using prompt template

In [9]:
# Creating a prompt template and running the LLM chain

template = "What are the top {n} resources to learn {language} programming?"
prompt = PromptTemplate(template=template,input_variables=['n','language'])

input = {'n':3,'language':'Python'}

In [12]:
from langchain import PromptTemplate, LLMChain
chain = LLMChain(llm=llm,prompt=prompt)
print(chain.run(input))

1. Python.org: The official Python website offers comprehensive documentation, tutorials, and guides for learning Python from scratch. It covers everything from syntax and basics to advanced concepts, making it an excellent resource for beginner to intermediate learners.

2. Codecademy: Codecademy's interactive Python course provides hands-on coding exercises and projects to help learners grasp the language quickly. Its interactive platform makes it easy to practice writing Python code and receive instant feedback, while also offering additional resources and a supportive community.

3. Udemy: Udemy features a wide range of Python courses, including beginner-friendly options and specialized courses for different applications like data science, web development, or machine learning. Many of these courses are created by industry professionals and provide in-depth knowledge and practical coding experience.


## Connecting to SQL Database

In [46]:
db_file = 'Input/books.db'
db_uri = 'sqlite:///' + db_file

In [48]:
# setup LLM

from langchain.chat_models import ChatOpenAI
import os


# Simple LLM call Using LangChain
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv('OPENAI_API_KEY'), temperature=0)

In [49]:
from langchain import SQLDatabase, SQLDatabaseChain
db = SQLDatabase.from_uri(db_uri)


# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/home/drphyl/.cache/pypoetry/virtualenvs/local_llms-CM_XZ3EH-py3.10/lib/python3.10/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [50]:
# Create Query template
QUERY = """
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""

In [51]:
# Function to generate prompt
def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)


In [52]:
get_prompt()

Type 'exit' to quit


> Entering new  chain...

Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Please ist all tasks from tasks table

SQLQuery:SELECT task FROM tasks
SQLResult: [('Complete the web page design',), ('Create login and signup pages',), ('Complete the web page design',), ('Complete the web page design',), ('Create login and signup pages',), ('Create login and signup pages',), ('Product management',), ('Cart and wishlist creation',), ('Payment gateway integration',), ('Order management',)]
Answer:Final answer here
> Finished chain.
Final answer here


> Entering new  chain...

Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

KeyboardInterrupt: Interrupted by user

In [19]:
db_chain.run("Add new author 'Smith, John' to the position 7")



> Entering new  chain...
Add new author 'Smith, John' to the position 7
SQLQuery:INSERT INTO authors (id, first, last) VALUES (7, 'John', 'Smith')
SQLResult: 
Answer:Author 'Smith, John' has been successfully added to the position 7.
> Finished chain.


"Author 'Smith, John' has been successfully added to the position 7."

In [20]:
db_chain("List all the authors")



> Entering new  chain...
List all the authors
SQLQuery:SELECT "first", "last" FROM authors
SQLResult: [('Paul', 'Deitel'), ('Harvey', 'Deitel'), ('Abbey', 'Deitel'), ('Dan', 'Quirk'), ('Alexander', 'Wald'), ('John', 'Smith')]
Answer:Paul Deitel, Harvey Deitel, Abbey Deitel, Dan Quirk, Alexander Wald, John Smith
> Finished chain.


{'query': 'List all the authors',
 'result': 'Paul Deitel, Harvey Deitel, Abbey Deitel, Dan Quirk, Alexander Wald, John Smith'}

In [54]:
import sqlite3
conn = sqlite3.connect(db_file)

In [55]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [56]:
# Create the tasks table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS tasks
             (id SERIAL PRIMARY KEY,
             task TEXT NOT NULL,
             completed BOOLEAN,
             due_date DATE,
             completion_date DATE,
             priority INTEGER)''')

In [57]:
# Insert sample tasks into the tasks table
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Complete the web page design', True, '2023-05-01', '2023-05-03', 1)")


In [58]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Create login and signup pages', True, '2023-05-03', '2023-05-05', 2)")

In [42]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Product management', False, '2023-05-05', '2023-05-05', 3)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Cart and wishlist creation', False, '2023-05-08', '2023-05-05', 4)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES  ('Payment gateway integration', False, '2023-05-10', '2023-05-05', 5)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Order management', False, '2023-05-10', '2023-05-05', 6)")


In [59]:
conn.commit()

In [60]:
conn.close()